In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import fit_library as fit_lib
import pandas as pd
%matplotlib nbagg
%reload_ext autoreload
%autoreload 2

In [7]:
# DATA
TH_HIGH = 1.414
TH_LOW = 0.404
Cint = 6.18E-12
GATE = 500E-9
pQ = 1E-12
Ileak = 3.72E-6

Qu = (TH_HIGH-TH_LOW)*Cint / pQ

In [8]:
pe_density = ["200pe","100pe","50pe","25pe"]
#pe_density = ["200pe"]
df = pd.DataFrame(columns=['pe_density','Q_true','Q_measured','Qu','N_flips','Q_residue'])



In [9]:
for pe in pe_density:
        for set in range(1,21):
            file = "_" + pe + "_500ns_set" + str(set) + ".txt"
            Q_total = np.array(pd.read_csv("./results/Qtotal"+ file ,header=None))
            residue=pd.read_csv("./results/residue"+ file,header=None,skiprows=2,sep="    ")
            counter=pd.read_csv("./results/counter"+ file,header=None,skiprows=2,sep="    ")
            residue = (residue.to_numpy()-TH_LOW)*Cint / pQ
            counter = counter.to_numpy()
                        
            residue_t = np.sum(residue[:,1])
            entero = np.array(counter[:,1], dtype='int')
            binario = np.array(entero[:], dtype='str')
            flips = 0
            for i in range(len(binario)):
                flips = flips + int(binario[i],2)
            q_meas = flips*Qu + residue_t - Ileak*GATE/pQ
            
            aux_pd = pd.DataFrame(data={'pe_density':[pe], 'Q_true':[Q_total[0,0]/pQ], 'Q_measured':[q_meas],
                                                'Qu':[Qu], 'N_flips':[flips], 'Q_residue':[residue_t]})
            df = df.append(aux_pd, ignore_index=True)

<ipython-input-9-dc937e51ae81>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  residue=pd.read_csv("./results/residue"+ file,header=None,skiprows=2,sep="    ")
<ipython-input-9-dc937e51ae81>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  counter=pd.read_csv("./results/counter"+ file,header=None,skiprows=2,sep="    ")


In [17]:
df[20:40]

,pe_density,Q_true,Q_measured,Qu,N_flips,Q_residue
20,100pe,105.3390,112.075185,6.2418,15,20.308185
21,100pe,106.2520,112.955902,6.2418,15,21.188902
22,100pe,104.0480,110.815620,6.2418,15,19.048620
23,100pe,106.6570,144.562808,6.2418,21,15.345008
24,100pe,98.7771,143.007543,6.2418,20,20.031543
25,100pe,94.8432,145.408071,6.2418,22,9.948471
26,100pe,100.9040,107.722549,6.2418,15,15.955549
27,100pe,103.5000,110.256528,6.2418,15,18.489528
28,100pe,102.7780,140.772503,6.2418,21,11.554703
29,100pe,100.7390,107.504308,6.2418,16,9.495508


In [203]:
int("110",2)

6

In [79]:
Q_total

array([[1.96097e-10]])